In [1]:
import psycopg as pg
import password

In [2]:
conn = pg.connect(dbname="postgres", host="localhost", user="postgres", password=password.password, port=5432)
cursor = conn.cursor()

## Список запросов к базе данных

In [3]:
requests = [
"""
/*
 * Выводим распределение (количество) клиентов по сферам деятельности,
 * отсортировав результат по убыванию количества.
*/
SELECT job_industry_category, COUNT(customer_id) AS num_of_customers
FROM customer c
GROUP BY job_industry_category
ORDER BY num_of_customers DESC;
""",
"""
/*
 * Находим сумму транзакций за каждый месяц по сферам деятельности,
 * отсортировав по месяцам и по сфере деятельности.
 */

SELECT EXTRACT(MONTH FROM TO_DATE(t.transaction_date, 'DD.MM.YYYY')) AS month
	,c.job_industry_category
	,SUM(t.list_price)
FROM "transaction" t
JOIN "customer" c ON t.customer_id = c.customer_id
GROUP BY c.job_industry_category, EXTRACT(MONTH FROM TO_DATE(t.transaction_date, 'DD.MM.YYYY'))
ORDER BY month, c.job_industry_category;
""",
"""
/*
 * Выводим количество онлайн-заказов для всех брендов в рамках
 * подтвержденных заказов клиентов из сферы IT.
 */
 SELECT t.brand
 	,COUNT(*) AS online_orders_approved
 FROM "transaction" t
 JOIN "customer" c ON c.customer_id = t.customer_id
 GROUP BY t.brand, t.online_order, t.order_status, c.job_industry_category
 HAVING t.online_order = 'True'
		AND t.order_status = 'Approved'
		AND c.job_industry_category = 'IT';
""",
"""
/*
 * Находим по всем клиентам сумму всех транзакций (list_price), максимум,
 * минимум и количество транзакций, отсортировав результат по убыванию
 * суммы транзакций и количества клиентов.
 */

-- Первый способ. Запрос с использованием GROUP BY
SELECT t.customer_id
	,SUM(t.list_price) AS total_amount
	,MAX(t.list_price) AS max_transaction
	,MIN(t.list_price) AS min_transaction
	,COUNT(t.list_price) AS transaction_count
FROM "transaction" t
GROUP BY t.customer_id
ORDER BY total_amount DESC, transaction_count DESC;
""",
"""
-- Второй способ. Запрос с использованием только оконных функций
SELECT DISTINCT t.customer_id
    ,SUM(t.list_price) OVER (PARTITION BY t.customer_id) AS total_amount
    ,MAX(t.list_price) OVER (PARTITION BY t.customer_id) AS max_transaction
    ,MIN(t.list_price) OVER (PARTITION BY t.customer_id) AS min_transaction
    ,COUNT(*) OVER (PARTITION BY t.customer_id) AS transaction_count
FROM
    "transaction" t
ORDER BY
    total_amount DESC, transaction_count DESC;
""",
"""
/* Находим имена и фамилии клиентов с минимальной/максимальной суммой транзакций
 * за весь период (сумма транзакций не может быть null). Пишем отдельные запросы
 * для минимальной и максимальной суммы.
 */

-- Выводим имена и фамилии клиентов с минимальной суммой транзакций
SELECT c.first_name, c.last_name
FROM "customer" c
JOIN "transaction" t ON c.customer_id  = t.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING SUM(t.list_price) = (
       	SELECT MIN(total_amount)
        FROM (SELECT SUM(list_price) AS total_amount
            FROM transaction
            GROUP BY customer_id) AS min_amount);
""",
"""
-- Выводим имена и фамилии клиентов с максимальной суммой транзакций
SELECT c.first_name, c.last_name
FROM "customer" c
JOIN "transaction" t ON c.customer_id  = t.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING SUM(t.list_price) = (
       	SELECT MAX(total_amount)
        FROM (SELECT SUM(list_price) AS total_amount
            FROM transaction
            GROUP BY customer_id) AS max_amount);
""",
"""
/*
 * Выводим только самые первые транзакции клиентов.
 * Решение с помощью оконных функций.
 */
SELECT DISTINCT t.customer_id
	,FIRST_VALUE (transaction_id) OVER (
	PARTITION BY customer_id ORDER BY TO_DATE(transaction_date, 'dd.mm.yyyy')
	) AS first_transaction
FROM "transaction" t
""",
"""
/*
 * Выводим имена, фамилии и профессии клиентов, между транзакциями которых был
 * максимальный интервал (интервал вычисляется в днях)
 */
WITH transaction_delta AS
(SELECT customer_id, MAX(delta) AS max_delta
FROM
(SELECT t.customer_id, TO_DATE(transaction_date, 'dd.mm.yyyy'), lead(TO_DATE(transaction_date, 'dd.mm.yyyy')) over(PARTITION BY t.customer_id)
,lead(TO_DATE(transaction_date, 'dd.mm.yyyy')) over(PARTITION BY t.customer_id
ORDER BY TO_DATE(transaction_date, 'dd.mm.yyyy')) - TO_DATE(transaction_date, 'dd.mm.yyyy') AS delta
FROM "transaction" t) AS a
GROUP BY customer_id)

SELECT c.first_name, c.last_name, c.job_title
FROM customer c
JOIN transaction_delta td ON td.customer_id = c.customer_id
WHERE td.max_delta = (SELECT MAX(max_delta) from transaction_delta)
"""
]

In [4]:
def request(n: int):
    r = requests[n]
    print(r)
    cursor.execute(r)
    answer = cursor.fetchall()
    for i in answer:
        print(*i)

### Задание 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества

In [5]:
request(0)


/*
 * Выводим распределение (количество) клиентов по сферам деятельности,
 * отсортировав результат по убыванию количества.
*/
SELECT job_industry_category, COUNT(customer_id) AS num_of_customers
FROM customer c
GROUP BY job_industry_category
ORDER BY num_of_customers DESC;

Manufacturing 799
Financial Services 774
n/a 656
Health 602
Retail 358
Property 267
IT 223
Entertainment 136
Argiculture 113
Telecommunications 72


### Задание 2. Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности

In [6]:
request(1)


/*
 * Находим сумму транзакций за каждый месяц по сферам деятельности,
 * отсортировав по месяцам и по сфере деятельности.
 */

SELECT EXTRACT(MONTH FROM TO_DATE(t.transaction_date, 'DD.MM.YYYY')) AS month
	,c.job_industry_category
	,SUM(t.list_price)
FROM "transaction" t
JOIN "customer" c ON t.customer_id = c.customer_id
GROUP BY c.job_industry_category, EXTRACT(MONTH FROM TO_DATE(t.transaction_date, 'DD.MM.YYYY'))
ORDER BY month, c.job_industry_category;

1 Argiculture 43513.812
1 Entertainment 64089.934
1 Financial Services 366383.78
1 Health 286860.44
1 IT 107783.414
1 Manufacturing 365232.38
1 n/a 316819.72
1 Property 100686.97
1 Retail 182375.72
1 Telecommunications 31210.2
2 Argiculture 60016.81
2 Entertainment 63965.99
2 Financial Services 375961.8
2 Health 268525.75
2 IT 93961.79
2 Manufacturing 389260.53
2 n/a 260711.9
2 Property 112255.41
2 Retail 146107.58
2 Telecommunications 27678.2
3 Argiculture 49048.492
3 Entertainment 77122.44
3 Financial Services 322268.28
3 Health 

### Задание 3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT

In [7]:
request(2)


/*
 * Выводим количество онлайн-заказов для всех брендов в рамках
 * подтвержденных заказов клиентов из сферы IT.
 */
 SELECT t.brand
 	,COUNT(*) AS online_orders_approved
 FROM "transaction" t
 JOIN "customer" c ON c.customer_id = t.customer_id
 GROUP BY t.brand, t.online_order, t.order_status, c.job_industry_category
 HAVING t.online_order = 'True'
		AND t.order_status = 'Approved'
		AND c.job_industry_category = 'IT';

None 8
Trek Bicycles 82
WeareA2B 90
Solex 101
Giant Bicycles 89
OHM Cycles 78
Norco Bicycles 92


### Задание 4. Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат. 

In [8]:
request(3)


/*
 * Находим по всем клиентам сумму всех транзакций (list_price), максимум,
 * минимум и количество транзакций, отсортировав результат по убыванию
 * суммы транзакций и количества клиентов.
 */

-- Первый способ. Запрос с использованием GROUP BY
SELECT t.customer_id
	,SUM(t.list_price) AS total_amount
	,MAX(t.list_price) AS max_transaction
	,MIN(t.list_price) AS min_transaction
	,COUNT(t.list_price) AS transaction_count
FROM "transaction" t
GROUP BY t.customer_id
ORDER BY total_amount DESC, transaction_count DESC;

2183 19071.322 2005.66 230.91 14
1129 18349.27 1992.93 290.62 13
1597 18052.68 2091.47 360.4 12
941 17898.459 2091.47 1057.51 10
2788 17258.94 2083.94 183.86 11
936 17160.24 2005.66 183.86 12
1887 17133.932 2091.47 688.63 11
1302 17035.83 1977.36 71.16 13
1140 16199.24 2083.94 183.86 13
2309 16122.341 2091.47 290.62 12
729 15825.999 2091.47 586.45 10
1103 15447.92 1977.36 230.91 12
1317 15370.81 2091.47 569.56 9
2874 15091.91 2005.66 544.05 11
2762 15071.26 1997.68 183.86 

In [9]:
request(4)


-- Второй способ. Запрос с использованием только оконных функций
SELECT DISTINCT t.customer_id
    ,SUM(t.list_price) OVER (PARTITION BY t.customer_id) AS total_amount
    ,MAX(t.list_price) OVER (PARTITION BY t.customer_id) AS max_transaction
    ,MIN(t.list_price) OVER (PARTITION BY t.customer_id) AS min_transaction
    ,COUNT(*) OVER (PARTITION BY t.customer_id) AS transaction_count
FROM
    "transaction" t
ORDER BY
    total_amount DESC, transaction_count DESC;

2183 19071.32 2005.66 230.91 14
1129 18349.27 1992.93 290.62 13
1597 18052.678 2091.47 360.4 12
941 17898.459 2091.47 1057.51 10
2788 17258.94 2083.94 183.86 11
936 17160.238 2005.66 183.86 12
1887 17133.93 2091.47 688.63 11
1302 17035.828 1977.36 71.16 13
1140 16199.241 2083.94 183.86 13
2309 16122.34 2091.47 290.62 12
729 15826.0 2091.47 586.45 10
1103 15447.92 1977.36 230.91 12
1317 15370.81 2091.47 569.56 9
2874 15091.911 2005.66 544.05 11
2762 15071.26 1997.68 183.86 10
255 14949.909 1810.0 441.49 12
1672 14908.461 19

### Задание 5. Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы

In [10]:
request(5)


/* Находим имена и фамилии клиентов с минимальной/максимальной суммой транзакций
 * за весь период (сумма транзакций не может быть null). Пишем отдельные запросы
 * для минимальной и максимальной суммы.
 */

-- Выводим имена и фамилии клиентов с минимальной суммой транзакций
SELECT c.first_name, c.last_name
FROM "customer" c
JOIN "transaction" t ON c.customer_id  = t.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING SUM(t.list_price) = (
       	SELECT MIN(total_amount)
        FROM (SELECT SUM(list_price) AS total_amount
            FROM transaction
            GROUP BY customer_id) AS min_amount);

Hamlen Slograve


In [11]:
request(6)


-- Выводим имена и фамилии клиентов с максимальной суммой транзакций
SELECT c.first_name, c.last_name
FROM "customer" c
JOIN "transaction" t ON c.customer_id  = t.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING SUM(t.list_price) = (
       	SELECT MAX(total_amount)
        FROM (SELECT SUM(list_price) AS total_amount
            FROM transaction
            GROUP BY customer_id) AS max_amount);

Jillie Fyndon


### Задание 6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций

In [12]:
request(7)


/*
 * Выводим только самые первые транзакции клиентов.
 * Решение с помощью оконных функций.
 */
SELECT DISTINCT t.customer_id
	,FIRST_VALUE (transaction_id) OVER (
	PARTITION BY customer_id ORDER BY TO_DATE(transaction_date, 'dd.mm.yyyy')
	) AS first_transaction
FROM "transaction" t

253 17514
854 6244
3220 10340
2737 15628
3188 9209
2657 9862
2254 7661
265 12086
2836 16957
2089 2524
1847 11377
1825 10263
1266 14489
2000 64
910 12296
3407 5509
2605 5896
3335 14762
1017 9280
121 17398
1456 18767
368 12703
1102 19977
3277 8073
3150 16910
2765 6881
1953 8767
1613 12416
1802 14842
1396 13587
2159 1740
1701 15475
2492 397
489 18066
1610 11703
2791 5533
1104 16181
228 5757
1532 10886
2123 15501
109 11704
2335 16295
3257 1102
626 1543
2037 3719
1312 8413
1484 12428
74 16590
3467 2667
1762 12782
2828 15733
2404 4497
3366 13812
400 6956
2400 15657
763 2787
28 7775
2345 8564
1016 1473
2752 11516
286 19209
1100 11779
2638 8030
2010 19915
1362 6364
2696 8782
2881 1331
683 12445
3375 3259
1619 89

### Задание 7. Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)

In [13]:
request(8)


/*
 * Выводим имена, фамилии и профессии клиентов, между транзакциями которых был
 * максимальный интервал (интервал вычисляется в днях)
 */
WITH transaction_delta AS
(SELECT customer_id, MAX(delta) AS max_delta
FROM
(SELECT t.customer_id, TO_DATE(transaction_date, 'dd.mm.yyyy'), lead(TO_DATE(transaction_date, 'dd.mm.yyyy')) over(PARTITION BY t.customer_id)
,lead(TO_DATE(transaction_date, 'dd.mm.yyyy')) over(PARTITION BY t.customer_id
ORDER BY TO_DATE(transaction_date, 'dd.mm.yyyy')) - TO_DATE(transaction_date, 'dd.mm.yyyy') AS delta
FROM "transaction" t) AS a
GROUP BY customer_id)

SELECT c.first_name, c.last_name, c.job_title
FROM customer c
JOIN transaction_delta td ON td.customer_id = c.customer_id
WHERE td.max_delta = (SELECT MAX(max_delta) from transaction_delta)

Susanetta None Legal Assistant
